<a href="https://colab.research.google.com/github/OswaldEkwoge/SD/blob/master/cat_or_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/Colab Notebooks/Cat or Dog/cats_and_dogs"

cat_or_dog.ipynb  dataset


In [8]:
"""Part 1 - Data Preprocessing"""

# Generating images for the Training Set ==> Using the keras documentation for Image Augmentation (using already-ready code)
# Type on browser "keras documentation" -> Allows us to enrich our dataset without adding more images, allowing us
# to get good results with little overfitting
# Use the code "flow from directory method"

# Generating images for the training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
# Generating images for the test set
test_datagen = ImageDataGenerator(rescale = 1./255)

# Creating the Trainig set
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Cat or Dog/cats_and_dogs/dataset/training_set',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')
# Creating the Test set
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/Cat or Dog/cats_and_dogs/dataset/test_set',
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode = 'binary')

Found 8027 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
"""Part 2 - Building the CNN"""

# Initializing the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution

# NB:kernel size is also called feature detector: the "3" means a 3x3 matrix, 32 feature detectors will be created
# remember we are working with a CPU, so 64 can be tricky.Therefore our CNN will be composed of 32 feature maps
# input_shape is the shape of our input images. Since they don't have the same format, we need to force them to have
# one single shape (pre-processing phase). So the expected format we choose is 64x64 (number of image dimensions), 
# the "3" is because images are colored.

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[64,64,3]))

In [0]:
# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

In [0]:
# Adding a second convolutional layer (to improve accuracy of results)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=[64,64,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))

In [0]:
# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())   # this results in our input layer on the artificial neural network (64 neurons input)

In [0]:
# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))   # adding a hidden layer of 128 neurons

In [0]:
# Step 5 - Output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # 1 output layer, binary (therefore the use of sigmoid)

In [0]:
""" Part 3 - Training the CNN"""

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [18]:
# Training the CNN on the Training set and evaluating it on the test set
cnn.fit_generator(training_set,
                 steps_per_epoch = 250,   # this equals number_of_samples / batch_size = 8000 / 32
                 epochs = 25,
                 validation_data = test_set,
                 validation_steps = 62)   # = 2000 / 32

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
250/250 [==============================] - 4739s 19s/step - loss: 0.6932 - accuracy: 0.5415 - val_loss: 0.6861 - val_accuracy: 0.5620
Epoch 2/25
250/250 [==============================] - 44s 174ms/step - loss: 0.6200 - accuracy: 0.6569 - val_loss: 0.6027 - val_accuracy: 0.6820
Epoch 3/25
250/250 [==============================] - 43s 172ms/step - loss: 0.5822 - accuracy: 0.6909 - val_loss: 0.5964 - val_accuracy: 0.7016
Epoch 4/25
250/250 [==============================] - 44s 176ms/step - loss: 0.5489 - accuracy: 0.7207 - val_loss: 0.5424 - val_accuracy: 0.7278
Epoch 5/25
250/250 [==============================] - 43s 173ms/step - loss: 0.5258 - accuracy: 0.7425 - val_loss: 0.5418 - val_accuracy: 0.7334
Epoch 6/25
250/250 [==============================] - 43s 173ms/step - loss: 0.5190 - accuracy: 0.7455 - val_loss: 0.5000 - val_accuracy: 0.7641
Epoch 7/25
250/250 [==============================] - 

In [22]:
"""Given an image, let's predict whether it is a cat or a dog"""

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os

def load_image(img_path, show=False):
    img = image.load_img(img_path, target_size=(64, 64))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255
    
    if show:
        plt.imshow(img_tensor[0])
        plt.axis('off')
        plt.show()
    return img_tensor

# Choose the image here
img_1 = '/content/drive/My Drive/Colab Notebooks/Cat or Dog/cats_and_dogs/dataset/test_image/dog_004.jpg'

img_2 = '/content/drive/My Drive/Colab Notebooks/Cat or Dog/cats_and_dogs/dataset/test_image/cat.4010.jpg'

# load a single image
new_img_1 = load_image(img_1)
new_img_2 = load_image(img_2)
images = np.vstack([new_img_1, new_img_2])
# check prediction
pred = cnn.predict(images)
print(pred)

[[0.58645725]
 [0.98779154]]


In [20]:
for  preds in pred:
    if preds < 0.5:
        print (f"{preds} -> cat")
    else:
        print (f"{preds} -> dog")

[0.98635626] -> dog
[0.7331336] -> dog
